# NASA demo template

In [1]:
import numpy as np

from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [2]:
from OpenVisus import *

Starting OpenVisus C:\Users\Vlaerio's PC\AppData\Roaming\Python\Python37\site-packages\OpenVisus\__init__.py 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)] sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0) ...


In [3]:
colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis','ocean', 'gist_earth', 'terrain', 'gist_stern',
             'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
             'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
             'gist_ncar']

In [4]:
def Assert(cond):
    if not cond:
        raise Exception("Assert failed")

class CachedDataset(PyDataset):
    
    # constructor
    def __init__(self, args):
        self.local_filename=os.path.abspath(args["local"]).replace("\\","/")
        self.remote_url=args["url"]
        self.remote_access_type = args["access"]
        self.description=args["description"]
        
        super().__init__(LoadDatasetCpp(self.remote_url))
        
        self.num_blocks = len(self.getFields()) * self.getTotalNumberOfBlocks() * len(self.getTimesteps().asVector())
        self.num_blocks_cached = 0

        self.stop_thread=False
        self.thread=None
        
        self.progress=None
        self.progress_display=None
        
    def __del__(self):
        self.stopCaching()   
        
    # createAccess
    def createAccess(self, ):
        
        access_config="""
            <access type='multiplex'>
                    <access type='disk' chmod='rw' url='file://{}' />
                    <access type='{}' url='{}' chmod="r" /> 
            </access>  
        """.format(
            self.local_filename.replace("&","&amp;"),
            self.remote_access_type,
            self.remote_url.replace("&","&amp;")) 
        
        # print("Creating access",access_config)

        access= self.createAccessForBlockQuery(StringTree.fromString(access_config))

        # at this point the cache is enabled with the new local idx file
        Assert(os.path.isfile(self.local_filename))

        return access   

    # startCaching
    def startCaching(self, background=True):
        
        if background:
            self.thread = threading.Thread(target=self.startCaching, args=(False,))
            self.stop_thread=False
            self.thread.start()        
            return 

        #print("start caching","...")
        
        access=self.createAccess()

        access.beginRead()
        
        for field in self.getFields():
            for blockid in range(self.getTotalNumberOfBlocks()): 
                for time in self.getTimesteps().asVector():
                    # print("Copying block","time",time,"field",field,"blockid",blockid,"...")
                    buffer =  self.readBlock(blockid, field=field, time=time, access=access)
                    
                     # to debug missing blocks
                    if  False and buffer is None :
                        read_block = db.createBlockQuery(blockid, ord('r'))
                        msg="# {} {} \n".format(blockid,read_block.getLogicBox().toString())
                        os.write(1, bytes(msg,'utf-8'))                   
                    
                    self.num_blocks_cached += 1
                    self.updateProgress()
                    if self.stop_thread:
                        # print("thread stopped")
                        access.endRead()
                        return
                        
        access.endRead()
        self.thread=None
        #print("caching finished done")
        
    # stopCaching
    def stopCaching(self):
        #print("stopping caching...")
        self.stop_thread=True
        if self.thread:
            self.thread.join()
            self.thread=None
    # getWidth
    def getWidth(self):
        p2=self.getLogicBox()[1]
        return p2[0]    
        
    # getHeight
    def getHeight(self):
        p2=self.getLogicBox()[1]
        return p2[1]   
        
    # getDepth
    def getDepth(self):
        p2=self.getLogicBox()[1]
        return p2[2]  
        
    # readSlice
    def readSlice(self,dir=0, slice=0,quality=-3, time=0, access=None):
        
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W] if dir!=0 else [slice,slice+1]
        y=[0,H] if dir!=1 else [slice,slice+1]
        z=[0,D] if dir!=2 else [slice,slice+1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        
        width,height=[value for value in ret.shape if value>1]
        return ret.reshape([width,height])
        
    # readColumn
    def readXYColumn(self,Height, Depth,quality=-3, time=0, access=None):
        W,H,D=self.getWidth(), self.getHeight(), self.getDepth()
        x=[0,W]
        y=[Height,Height+1]
        z=[Depth ,Depth +1] 
        ret=self.read(x=x, y=y,z=z, quality=quality,time=time,access=access)
        #print(">",ret.shape)
        width=[value for value in ret.shape if value>1]
        return ret
        
    # setProgress
    def setProgress(self,progress, progress_display):
        self.progress=progress
        self.progress_display=progress_display   
        self.progress.min=0
        self.progress.max =self.num_blocks       

    # updateProgress
    def updateProgress(self):
                    
        if self.progress:
            self.progress.value = self.num_blocks_cached

        if self.progress_display:
            self.progress_display.value = (
                "Caching progress %.2f%% (%d/%d)" % (
                    100 * self.num_blocks_cached/self.num_blocks, 
                    self.num_blocks_cached,
                    self.num_blocks))                    

print("Utilities defined")

Utilities defined


In [5]:
local_cache="./visus-cache/foam/visus.idx"
MettalicFoam =    {
        "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=foam-2022-01&cached=1",
        "access":"network",
        "local": local_cache,
        "description":'University of Utah Campus Server'
    }
    
# function to plot the image data with matplotlib
def ShowData(data, cmap=None, plot=None,width = 6):
    ratio=float(data.shape[1])/data.shape[0]
    x = 100
    y = float(data.shape[1])/data.shape[0]*x
    plot = figure(plot_height=y, plot_width=y)
    theImage = plot.image(image=[data], x=0, y=0, dw=x, dh=y, palette="Spectral11", level="image")
    return plot

db=CachedDataset(MettalicFoam )
access=db.createAccess()
first_query=db.readSlice(dir=2, slice=512, access=access, time=0, quality=-3)

In [6]:
dbWidth  = db.getWidth()
dbHeight = db.getHeight()
dbDepth  = db.getDepth()
print(db.getWidth())
print(db.getHeight())
print(db.getDepth())


1055
1024
1024


In [7]:
a = db.readSlice(dir=0, slice=500, time=0)
print(a)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
import colorsys
import matplotlib
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar
from bokeh.palettes import *


Points = [
  -10,
  -2.18552,
  -0.260014,
  0.00702716,
  0.372452,
  0.709768,
  10  
]

RGBPoints = [  
  [0.27843137254900002, 0.27843137254900002, 0.85882352941200002],
  [0.0, 0.33333333333333331, 1.0],
  [0.33333333333333331, 0.66666666666666663, 1.0],
  [0.0, 0.0, 0.49803921568627452],  
  [0.82745098039215681, 1.0, 0.73333333333333328],
  [1.0, 0.66666666666666663, 0.0],
  [0.87843137254899994, 0.30196078431399997, 0.30196078431399997]
]

def get_continuous_cmap(rgb_list, float_list=None):
  if float_list:
    # normalize the float list
    min_val = min(float_list)
    max_val = max(float_list)
    my_range = max_val - min_val
    float_list = [(v - min_val)/my_range for v in float_list]
    print(float_list)
  else:
    float_list = list(np.linspace(0, 1, len(rgb_list)))

  cdict = dict()  
  for num, col in enumerate(['red', 'green', 'blue']):
    col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(0, len(float_list))]
    cdict[col] = col_list
  cmp = matplotlib.colors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
  #color_mapper = LinearColorMapper(palette=palette, low=-10, high=10)
  return cmp

my_cmap = get_continuous_cmap(RGBPoints, Points)

my_cmap = LinearColorMapper(palette="Turbo256", low=-10, high=10)
my_cmap2 = LinearColorMapper(palette="Turbo256", low=-10, high=10)


[0.0, 0.39072399999999996, 0.4869993, 0.500351358, 0.5186225999999999, 0.5354884, 1.0]


In [9]:
print(np.min(a))
print(np.max(a))
my_cmap = LinearColorMapper(palette="Turbo256", low=np.min(a), high=np.max(a))
my_cmap1 = LinearColorMapper(palette="Turbo256", low=np.min(a), high=np.max(a))
my_cmap2 = LinearColorMapper(palette="Turbo256", low=np.min(a), high=np.max(a))

0
60669


In [10]:
%%time
tmpVerticalImage = np.zeros((528, 18))
def getVerticalImage(x,y,db=db, access=access):
    global tmpVerticalImage
    for i in range(0,18,1):
        col =db.readXYColumn((x//2)*2,(y//2)*2,time=i, access=access)
        #col =db.readXYColumn(x,y,time=i, access=access)
        #print(col.shape)
        tmpVerticalImage[:,i]=col
    return ( tmpVerticalImage)


Wall time: 0 ns


In [11]:
import numpy as np

from bokeh.plotting import figure, show

N = 500
x = np.linspace(0, 5, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)
#myImage.data_source.data  = {"image" :[d]}
counter = 0

db=CachedDataset(MettalicFoam )
access=db.createAccess()
first_query=db.readSlice(dir=2, slice=512, access=access, time=0, quality=-3)

# Here should be the code that returns an image
def getImage(xDim,yDim):
    N = 500
    x = np.linspace(0, xDim, N)
    y = np.linspace(0, yDim, N)
    xx, yy = np.meshgrid(x, y)
    d = np.sin(xx)*np.cos(yy)
    return d
    
def getHorizontalImage(depth,time,db=db, access=access):
    return db.readSlice(dir=0, slice=(depth//8)*8, time=time,quality=-3)
    
for i in range(0,18,3):
    col =db.readXYColumn((500//2)*2,(500//2)*2,time=i,access=access)
    #print(col.shape)
    tmpVerticalImage[:,i]=col
    tmpVerticalImage[:,i+1]=col
    tmpVerticalImage[:,i+2]=col
    
dbWidth  = db.getWidth()
dbHeight = db.getHeight()
dbDepth  = db.getDepth()
needsRedraw = True

def modify_doc(doc):
    global tmpVerticalImage
    # Set up data
    wDim = 360
    hDim = 180
    xMin = 0
    xMax = 1024
    yMin = 0
    yMax = 1024
    yDim = 20
    scale = 0.3
    wDim = int((xMax-xMin)*scale)
    hDim = int((yMax-yMin)*scale)
    
    N = 200
    x = np.linspace(0, 4*np.pi, N)
    y = np.sin(x)
    source = ColumnDataSource(data=dict(x=x, y=y))

    # Set up plot
    plot = figure(plot_height=hDim, plot_width=wDim, 
                  title="Select longitude and latiutude",
                  x_axis_label='Longitude',
                  y_axis_label='Latitude',
                  x_range=[0, 1024], y_range=[0, 1024], toolbar_location=None)
    plot.xgrid[0].grid_line_color=None
    plot.ygrid[0].grid_line_color=None
    
    #theImage = plot.image(image=[d], x=xMin, y=yMin, dw=xMax-xMin, dh=yMax-yMin, palette="Turbo256", level="image")
    d = getHorizontalImage(500,8)
    theImage = plot.image(image=[d], x=xMin, y=yMin, dw=xMax-xMin, dh=yMax-yMin, color_mapper =my_cmap1, level="image")

    # Set up plot
    plot2 = figure(plot_height=hDim, plot_width=wDim, 
                  x_axis_label='Time',
                  y_axis_label='Depth',
                  x_range=[0, 18], y_range=[0, 1055], toolbar_location=None)
    plot2.xgrid[0].grid_line_color=None
    plot2.ygrid[0].grid_line_color=None

    theImage2 = plot2.image(image=[tmpVerticalImage], 
                            x=0, y=0, dw=18, dh=1055, color_mapper =my_cmap2, level="image")

    xSource = ColumnDataSource()
    ySource = ColumnDataSource()
    line_color1="#f46d43"
    line_color="#ffffff"
    line_width=3
    line_width2=1
    plot.line('x', 'y', source=xSource, line_width=line_width,line_color=line_color)
    plot.line('x', 'y', source=ySource, line_width=line_width,line_color=line_color)
    plot.line('x', 'y', source=xSource, line_width=line_width2,line_color=line_color1)
    plot.line('x', 'y', source=ySource, line_width=line_width2,line_color=line_color1)

    dSource = ColumnDataSource()
    tSource = ColumnDataSource()
    plot2.line('x', 'y', source=dSource, line_width=line_width,line_color=line_color)
    plot2.line('x', 'y', source=tSource, line_width=line_width,line_color=line_color)
    plot2.line('x', 'y', source=dSource, line_width=line_width2,line_color=line_color1)
    plot2.line('x', 'y', source=tSource, line_width=line_width2,line_color=line_color1)
    
    # Set up widgets
    xPosition = Slider(title="Longitude", value=500, start=0, end=1000, step=1,width=wDim)
    yPosition = Slider(title="Latitude", value=500, start=0, end=1000, step=1, width=wDim)

    depth = Slider(title="Depth", value=dbWidth//2, start=0, end=dbWidth, step=1,width=wDim)
    timeStep = Slider(title="timeStep", value=0, start=0, end=17, step=1, width=wDim)
    
    def setValues():
        xSource.data = dict(x=[xPosition.value,xPosition.value], y=[yMin, yMax])
        ySource.data = dict(x=[xMin, xMax], y=[yPosition.value,yPosition.value])
        
        dSource.data = dict(x=[0,18], y=[depth.value, depth.value])
        tSource.data = dict(x=[timeStep.value, timeStep.value], y=[0,1055])
        
    setValues()

    text = TextInput(title="title", value='my sine wave')
    offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)

    amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
    phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
    freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)

    # Set up callbacks
#     def update_title(attrname, old, new):
#         plot.title.text = text.value

#     text.on_change('value', update_title)

    #Not using events. Just update asyncronosly
    def update_data(attrname, old, new):
        pass
        #setValues()


    #Not using events. Just update asyncronosly
#     for w in [xPosition,yPosition,offset, amplitude, phase, freq]:
#         w.on_change('value', update_data)


    myWidgets = column(xPosition, yPosition, depth,timeStep) 
    myGrid = row(myWidgets,plot,plot2)
    doc.add_root(myGrid)
    
    doc.title = "NASA demo"

    needsRedraw = True
    def asyncUpdate():
        global counter
        global tmpVerticalImage
        global needsRedraw  
        a = tmpVerticalImage
        if not hasattr(asyncUpdate, "timeStepOld"):
            asyncUpdate.timeStepOld  = timeStep.value    
            asyncUpdate.xPositionOld = xPosition.value
            asyncUpdate.yPositionOld = yPosition.value
            asyncUpdate.depthOld     = depth.value

        #flush all events until steady
        if (asyncUpdate.timeStepOld  != timeStep.value   or    
            asyncUpdate.xPositionOld != xPosition.value  or
            asyncUpdate.yPositionOld != yPosition.value  or
            asyncUpdate.depthOld     != depth.value):
            
            asyncUpdate.timeStepOld  = timeStep.value    
            asyncUpdate.xPositionOld = xPosition.value
            asyncUpdate.yPositionOld = yPosition.value
            asyncUpdate.depthOld     = depth.value
            needsRedraw = True
            return
            
        if needsRedraw:
            setValues()
            theImage.data_source.data   = {"image" :[getHorizontalImage(depth.value    ,timeStep.value)]}
            a = getVerticalImage  (xPosition.value,yPosition.value)
            theImage2.data_source.data  = {"image" :[a]}
            needsRedraw = False

        counter +=1
        print(counter, xPosition.value,yPosition.value,depth.value,timeStep.value,
              end = '                                    \r')        
    
    doc.add_periodic_callback(asyncUpdate, 100)


In [12]:
show(modify_doc)